In [9]:
# Environment setup and module import
from __future__ import print_function
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float

In [2]:
# Import DF from CSV

train_file = 'data/train_data_clean.csv'
train_label_file = 'data/train_labels.csv'
train_df = pd.read_csv(train_file, low_memory=False)
train_label = pd.read_csv(train_label_file, low_memory=False)


In [41]:
# Perform whatever conversions are necessary to get everything into proper format: ndarray

le = preprocessing.LabelEncoder()
train2 = train_df[train_df.select_dtypes(object).columns].apply(le.fit_transform)
for c in train_df[train_df.select_dtypes(int).columns]:
    train2[c] = train_df[c]
for c in train_df[train_df.select_dtypes(bool).columns]:
    train2[c] = train_df[c]

# To do: Perform standardization for input parameters


# Assign correct labels
x_train, y_train = train2, train_label



In [42]:
# Testing Cell
print(x_train.shape)
print(y_train.shape)

(8921483, 97)
(8921483, 2)


In [44]:
# Pytorch setup

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 81, x_train.shape[1], 100, 2

# Create random input and output data
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

# To do: figure out appropriate learning rate
learning_rate = 1e-5

# To do: make sure formatting is correct
#x_train_tensor = torch.from_numpy(x_train).float().to(device)
#y_train_tensor = torch.from_numpy(y_train).float().to(device)



TypeError: expected np.ndarray (got DataFrame)

In [12]:
# Placeholder for future operations
#for t in range(500):
#    # Forward pass: compute predicted y
#    h = x.mm(w1)
#    h_relu = h.clamp(min=0)
#    y_pred = h_relu.mm(w2)
#
#    # Compute and print loss
#    loss = (y_pred - y).pow(2).sum().item()
#    if t % 100 == 99:
#        print(t, loss)
#
#    # Backprop to compute gradients of w1 and w2 with respect to loss
#    grad_y_pred = 2.0 * (y_pred - y)
#    grad_w2 = h_relu.t().mm(grad_y_pred)
#    grad_h_relu = grad_y_pred.mm(w2.t())
#    grad_h = grad_h_relu.clone()
#    grad_h[h < 0] = 0
#    grad_w1 = x.t().mm(grad_h)
#
#    # Update weights using gradient descent
#    w1 -= learning_rate * grad_w1
#    w2 -= learning_rate * grad_w2